In [1]:
import ray
#import ray.services
from ray import workflow
from typing import List

if not ray.is_initialized():
    ray.init(address="auto")


In [5]:
from ray import workflow
from typing import List

@workflow.step
def read_data(num: int):
    return [i for i in range(num)]

@workflow.step
def preprocessing(data: List[float]) -> List[float]:
    return [d**2 for d in data]

@workflow.step
def aggregate(data: List[float]) -> float:
    return sum(data)


if not 'workflow_init' in vars():
    workflow_init = True
    workflow.init(storage="/tmp/data")

# Initialize workflow storage.
# workflow.init()

# Setup the workflow.
data = read_data.step(10)
preprocessed_data = preprocessing.step(data)
output = aggregate.step(preprocessed_data)

# Execute the workflow and print the result.
# print(output.run())

# The workflow can also be executed asynchronously.
print(ray.get(output.run_async()))

2022-04-29 07:22:23,357	INFO execution.py:59 -- Workflow job created. [id="27c13ebf-54f2-43c3-9108-06d49eae66f8.1651188143.356632948", storage_url="file:///tmp/data"]. Type: FUNCTION.


RuntimeError: This event loop is already running

sys:1: RuntimeWarning: coroutine 'WorkflowStorage._put' was never awaited
(raylet) [2022-04-29 19:24:08,603 E 30794 31115] (raylet) agent_manager.cc:107: The raylet exited immediately because the Ray agent failed. The raylet fate shares with the agent. This can happen because the Ray agent was unexpectedly killed or failed. See `dashboard_agent.log` for the root cause.


In [2]:
from ray import workflow

if not 'workflow_init' in vars():
    workflow_init = True
    workflow.init(storage="/tmp/data")

@workflow.step
def one() -> int:
    return 1

@workflow.step
def add(a: int, b: int) -> int:
    return a + b

output: "Workflow[int]" = add.step(100, one.step())

2022-04-29 07:13:05,342	INFO api.py:60 -- Using storage: /tmp/data


In [3]:
ref = output.run_async()

## or ??
## assert output.run(workflow_id="run_1") == 101

###  This event loop is already running ????
## assert workflow.get_status("run_1") == workflow.WorkflowStatus.SUCCESSFUL
##assert workflow.get_output("run_1") == 101

2022-04-29 07:13:07,613	INFO execution.py:59 -- Workflow job created. [id="a3963d9f-4068-4f1d-8c76-b9f02147526e.1651187587.612340689", storage_url="file:///tmp/data"]. Type: FUNCTION.


RuntimeError: This event loop is already running

In [4]:
@workflow.step
def read_data(num: int):
    return [i for i in range(num)]

@workflow.step
def preprocessing(data: List[float]) -> List[float]:
    return [d**2 for d in data]

@workflow.step
def aggregate(data: List[float]) -> float:
    return sum(data)


In [5]:
ref = output.run_async()

2022-04-29 07:13:25,172	INFO execution.py:59 -- Workflow job created. [id="ad62d0e9-9dc9-4834-809c-d163ec77e773.1651187605.172235012", storage_url="file:///tmp/data"]. Type: FUNCTION.


RuntimeError: This event loop is already running

In [6]:
@workflow.step
def add(left: int, right: int) -> int:
    return left + right

@workflow.step
def get_val() -> int:
    return 10

ret = add.step(get_val.step(), 20)
#assert ret.run("add_example") == 30


In [7]:
ret.run("add example")


2022-04-29 07:13:35,307	INFO execution.py:59 -- Workflow job created. [id="add example", storage_url="file:///tmp/data"]. Type: FUNCTION.


RuntimeError: This event loop is already running

In [8]:
@workflow.step
def double(v):
    return 2 * v

inner_step = double.options(name="inner").step(1)
outer_step = double.options(name="outer").step(inner_step)
result = outer_step.run_async("double")
inner = workflow.get_output("double", name="inner")
outer = workflow.get_output("double", name="outer")

assert ray.get(inner) == 2
assert ray.get(outer) == 4
assert ray.get(result) == 4

2022-04-29 07:13:39,658	INFO execution.py:59 -- Workflow job created. [id="double", storage_url="file:///tmp/data"]. Type: FUNCTION.
/home/ziyu4huang/py37/lib/python3.7/inspect.py:2896: RuntimeWarning: coroutine 'WorkflowStorage._put' was never awaited
  param = next(parameters)


RuntimeError: This event loop is already running

In [2]:

assert output.run(workflow_id="run_1") == 101
assert workflow.get_status("run_1") == workflow.WorkflowStatus.SUCCESSFUL
assert workflow.get_output("run_1") == 101

NameError: name 'output' is not defined

In [9]:
@ray.remote
def hello():
    return "hello"

@workflow.step
def words() -> List[ray.ObjectRef]:
    return [hello.remote(), ray.put("world")]

@workflow.step
def concat(words: List[ray.ObjectRef]) -> str:
    return " ".join([ray.get(w) for w in words])

#workflow.init()
#assert concat.step(words.step()).run() == "hello world"

concat.step(words.step()).run()

2022-04-29 07:13:43,808	INFO execution.py:59 -- Workflow job created. [id="c51ea7eb-40f6-4d3e-90e2-dc9f06093ccd.1651187623.807679176", storage_url="file:///tmp/data"]. Type: FUNCTION.


RuntimeError: This event loop is already running

In [5]:
concat.step(words.step()).run()

2022-04-22 22:10:25,477	INFO execution.py:49 -- Workflow job created. [id="4edeaf1a-0ec2-4259-b310-39a6916c0b2a.1650636625.476421833", storage_url="file:///tmp/data"]. Type: FUNCTION.


RuntimeError: This event loop is already running

In [11]:
from ray import workflow
import ray

@workflow.virtual_actor
class Counter:
    def __init__(self, init_val):
        self._val = init_val

    def incr(self, val=1):
        self._val += val
        print(self._val)

    @workflow.virtual_actor.readonly
    def value(self):
        return self._val

#workflow.init()


In [12]:

# Initialize a Counter actor with id="my_counter".
counter = Counter.get_or_create("my_counter", 0)

# Similar to workflow steps, actor methods support:
# - `run()`, which will return the value
# - `run_async()`, which will return a ObjectRef
counter.incr.run(10)
assert counter.value.run() == 10

# Non-blocking execution.
counter.incr.run_async(10)
counter.incr.run(10)
assert 30 == ray.get(counter.value.run_async())

RuntimeError: This event loop is already running

In [13]:
from ray import workflow
import time

# Create an event which finishes after 60 seconds.
event1_step = workflow.wait_for_event(workflow.event_listener.TimerListener, time.time() + 60)

# Create another event which finishes after 30 seconds.
event2_step = workflow.wait_for_event(workflow.event_listener.TimerListener, time.time() + 30)

@workflow.step
def gather(*args):
    return args;

# Gather will run after 60 seconds, when both event1 and event2 are done.
gather.step(event1_step, event2_step).run()

/home/ziyu4huang/tf1dml/lib/python3.7/site-packages/ray/_private/signature.py:68: RuntimeWarning: coroutine 'WorkflowStorage._put' was never awaited
  signature_parameters = list(get_signature(func).parameters.values())
2022-04-29 07:14:07,325	INFO execution.py:59 -- Workflow job created. [id="4c9b3a86-bce3-4c0f-984d-f54b484b3eab.1651187647.325121641", storage_url="file:///tmp/data"]. Type: FUNCTION.


RuntimeError: This event loop is already running